# Qwen2.5VL Inference


https://www.analyticsvidhya.com/blog/2025/01/qwen2-5-vl-vision-model/

In [7]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor

# Load the model
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)

# Load the processor for handling inputs (images, text, etc.)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct", force_download=True)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.20s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "images/Lor_Mee_3.jpg"},
            {"type": "text", "text": "Describe this image."}
        ]
    }
]


In [9]:
# Process the messages (images + text)
from qwen_vl_utils.vision_process import process_vision_info
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")  # Move the input to GPU if available


In [10]:
# Generate the output from the model
generated_ids = model.generate(**inputs, max_new_tokens=128)

# Decode the output
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)

print(output_text)


OutOfMemoryError: CUDA out of memory. Tried to allocate 183.52 GiB. GPU 0 has a total capacity of 15.99 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 18.44 GiB is allocated by PyTorch, and 365.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import dashscope

# Set your Dashscope API key
dashscope.api_key = "your_api_key"

# Make the API call with the desired model and messages
response = dashscope.MultiModalConversation.call(
    model='qwen2.5-vl-72b-instruct',
    messages=[{"role": "user", "content": [{"image": "image_url"}, {"text": "Query"}]}]
)

# You can access the response here
print(response)
